<a href="https://colab.research.google.com/github/olanigan/dspy-transliterate/blob/master/YoutubeAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OpenAI Whisper

In [ ]:
!pip install -q openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 21.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.2 MB/s eta 0:00:00


In [1]:
pip install --no-deps -U yt-dlp youtube-dl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.3 MB/s eta 0:00:00


In [2]:
url = "https://www.youtube.com/watch?v=ht3N2uZWzTw"


In [3]:
# url = short
!yt-dlp --no-config -v --extract-audio --audio-format mp3 $url -o audio.mp3 --convert-subs srt

[debug] Command-line config: ['--no-config', '-v', '--extract-audio', '--audio-format', 'mp3', 'https://www.youtube.com/watch?v=ht3N2uZWzTw', '-o', 'audio.mp3', '--convert-subs', 'srt']
[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out utf-8, error utf-8, screen utf-8
[debug] yt-dlp version stable@2024.10.07 from yt-dlp/yt-dlp [1a176d874] (pip)
[debug] Python 3.10.12 (CPython x86_64 64bit) - Linux-6.1.85+-x86_64-with-glibc2.35 (OpenSSL 3.0.2 15 Mar 2022, glibc 2.35)
[debug] exe versions: ffmpeg 4.4.2 (setts), ffprobe 4.4.2
[debug] Optional libraries: certifi-2024.08.30, requests-2.32.3, secretstorage-3.3.1, sqlite3-3.37.2, urllib3-2.2.3
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}
[debug] Request Handlers: urllib, requests
[debug] Loaded 1838 extractors
[youtube] Extracting URL: https://www.youtube.com/watch?v=ht3N2uZWzTw
[youtube] ht3N2uZWzTw: Downloading webpage
[youtube] ht3N2uZWzTw: Downloading ios player API JSON
[youtube] ht3N2uZWzTw: D

## GROQ API

In [4]:
!pip install -q groq pydub
import logging
from groq import Groq
from google.colab import userdata
from pydub import AudioSegment
import math
import os
import yt_dlp

def transcribe(video_url):
    # video_url = request.json["video_url"]
    # language = request.json.get("language", "en")
    # logging.info(f"Received transcription request for URL: {video_url} in language: {language}")

    # platform = get_video_platform(video_url)
    # if platform == 'unknown':
    #     return jsonify({"error": "Unsupported video platform"}), 400

    try:
        logging.info("Attempting to download audio")
        audio_file = "audio.mp3"
        logging.info(f"Audio downloaded successfully: {audio_file}")

        logging.info("Initializing Groq client")
        client = Groq(api_key=userdata.get('GROQ_API_KEY'))

        logging.info("Loading audio file")
        audio = AudioSegment.from_mp3(audio_file)

        chunk_length_ms = 10 * 60 * 1000
        num_chunks = math.ceil(len(audio) / chunk_length_ms)
        logging.info(f"Audio will be processed in {num_chunks} chunks")

        transcriptions = []

        for i in range(num_chunks):
            logging.info(f"Processing chunk {i+1}/{num_chunks}")
            start_time = i * chunk_length_ms
            end_time = min((i + 1) * chunk_length_ms, len(audio))

            chunk = audio[start_time:end_time]
            chunk_file = f"chunk_{i}.mp3"
            chunk.export(chunk_file, format="mp3")

            logging.info(f"Transcribing chunk {i+1}/{num_chunks}")
            with open(chunk_file, "rb") as file:
                transcription = client.audio.transcriptions.create(
                    file=(chunk_file, file.read()),
                    model="distil-whisper-large-v3-en",
                    response_format="text",
                    language="en",
                    temperature=0.0
                )
            transcriptions.append(transcription)
            logging.info(f"Chunk {i+1} transcription: {transcription[:100]}...")

            os.remove(chunk_file)

        full_transcription = "\n\n".join(transcriptions)
        logging.info(f"Full transcription (first 500 chars): {full_transcription[:500]}...")

        transcript_file = f"{os.path.splitext(os.path.basename(audio_file))[0]}_transcript.txt"
        with open(transcript_file, "w") as f:
            f.write(full_transcription)

        logging.info("Transcription completed successfully")

        # new_transcription = Transcription(
        #     video_url=video_url,
        #     transcription_text=full_transcription,
        #     audio_file=audio_file,
        #     transcript_file=transcript_file,
        #     language=language
        # )
        # db.session.add(new_transcription)
        # db.session.commit()
        # logging.info("Transcription saved to database")

        # return jsonify({
        #     "transcription": full_transcription,
        #     "audio_file": audio_file,
        #     "transcript_file": transcript_file,
        #     "language": language,
        #     "transcription_id": new_transcription.id
        # })
    except Exception as e:
        logging.error(f"Error during transcription: {str(e)}")
        return jsonify({"error": str(e)}), 500


def download_youtube_audio(youtube_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': '%(id)s.%(ext)s',
        'ffmpeg_location': '/nix/store/3zc5jbvqzrn8zmva4fx5p0nh4yy03wk4-ffmpeg-6.1.1-bin/bin/ffmpeg',
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        return f"{info['id']}.mp3"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [5]:
transcribe("")

## Miscellaneous

In [ ]:
# prompt: Trim audio.mp3 to first 15 minutes

!ffmpeg -i audio.mp3 -ss 0 -t 900 output.mp3


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
!ls /content -la

total 32072
drwxr-xr-x 1 root root     4096 Apr 16 12:25 .
drwxr-xr-x 1 root root     4096 Apr 16 12:17 ..
-rw-r--r-- 1 root root 18420285 Jun 22  2022 audio.mp3
drwxr-xr-x 4 root root     4096 Apr 12 13:21 .config
-rw-r--r-- 1 root root 14400621 Apr 16 12:25 output.mp3
drwxr-xr-x 1 root root     4096 Apr 12 13:22 sample_data


In [ ]:
# prompt: Convert output.mp3 to AMR format with codec amr_nb encoder

!ffmpeg -i output.mp3 -ar 8000 -ac 1 -c:a amr_nb_encoder output.amr


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# prompt: Download Youtube video as mp4

# !yt-dlp -f 'bestvideo[ext=mp4]' $url -o babies.mp4


In [ ]:
# prompt: Print metadata of babies.mp4 video including resolution like 4K, 2K etc

import subprocess

# Get the metadata of the video using ffprobe
metadata = subprocess.check_output(['ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', '/content/babies.mp4'])

# Parse the metadata to extract the resolution
import json
metadata_dict = json.loads(metadata)
# resolution = metadata_dict['format']['width'] + 'x' + metadata_dict['format']['height']

# Print the resolution
print(metadata_dict)


{'format': {'filename': '/content/babies.mp4', 'nb_streams': 1, 'nb_programs': 0, 'format_name': 'mov,mp4,m4a,3gp,3g2,mj2', 'format_long_name': 'QuickTime / MOV', 'start_time': '0.000000', 'duration': '19.699000', 'size': '7622373', 'bit_rate': '3095537', 'probe_score': 100, 'tags': {'major_brand': 'dash', 'minor_version': '0', 'compatible_brands': 'iso6mp41', 'creation_time': '2024-02-26T16:23:00.000000Z'}}}


In [ ]:
#import whisper
# model = whisper.load_model('base.en')
# url = "https://www.youtube.com/watch?v=FX0lBqeHuZo"
# url = "https://www.youtube.com/watch?v=W3BkJ6UxfvQ"
# short = "https://www.youtube.com/shorts/sD_GxH6VjWc"

In [ ]:
import whisper
# smodel = whisper.load_model('small')
wmodel = whisper.load_model('medium')

# model = whisper.load_model("large-v3")
# res1 = model.transcribe("audio.mp3")

100%|█████████████████████████████████████| 2.88G/2.88G [00:57<00:00, 53.6MiB/s]


In [ ]:
!ffmpeg

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
def export_text_to_file(text, filename):
  """Exports the given text to the given file and force ascii."""
  text = text.encode("ascii", errors="ignore").decode()
  with open(filename, 'w') as f:
    f.write(text)
# export_text_to_file(result['text'],"transcript.txt")

In [ ]:
# %%time
result1 = model.transcribe("audio.mp3")

export_text_to_file(result1['text'],"transcript_23_17.txt")

NameError: name 'model' is not defined

In [ ]:
# result['text'][:-2000]
# export_text_to_file(result1['text'],"transcript_12_16.txt")
print(f"Transcription:\n\n {result1['text'][:2000]}\n\n\n{result1['text'][-2000:]}")

Transcription:

  All right, folks. Oh, no, I haven't done the thing yet. One second. Oh. You know this. Oh, it's got batteries, yeah. Where's the batteries? It's booked. Blah, blah, blah. Don't pause. You should check with your booking manager. How did that battery go, man? That's crazy. I am, but I'm You know who's important that we don't consider to be important? The people who are on podcasts. What's going on, man? Yeah, okay. You don't have to make me hear it. Yeah. All right. There we go. All right. Bismillah ar-Rahman ar-Raheem. Alhamdulillah. Rabbil Alameen. Allahumma salli wa sallim wa barak la nabiyina Muhammad wa ala alihi wa sahbihi ajma'in. Allahumma la sahlah illa ma ja'al tau sahlah wa anta tajulul hazna. Iza shaytah sahlah. Allahumma a'inna ala zikrika wa shukrika wa husna ibadatika ya Rabbil Karim. Assalamu alaikum wa rahmatullah wa barakatuhu. Everybody here in Makkah al-Mukarramah. Everybody here in Makkah al-Mukarramah. Everybody here in Makkah al-Mukarramah. Everyb

In [ ]:
result1['segments'][:3]
# export_text_to_file(result1['segments'],"segments.txt")

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 3.06,
  'text': ' All right, folks.',
  'tokens': [50365, 1057, 558, 11, 4024, 13, 50518],
  'temperature': 0.0,
  'avg_logprob': -0.35171220952814275,
  'compression_ratio': 1.2300884955752212,
  'no_speech_prob': 0.35816094279289246},
 {'id': 1,
  'seek': 0,
  'start': 3.48,
  'end': 4.9,
  'text': " Oh, no, I haven't done the thing yet.",
  'tokens': [50539,
   876,
   11,
   572,
   11,
   286,
   2378,
   380,
   1096,
   264,
   551,
   1939,
   13,
   50610],
  'temperature': 0.0,
  'avg_logprob': -0.35171220952814275,
  'compression_ratio': 1.2300884955752212,
  'no_speech_prob': 0.35816094279289246},
 {'id': 2,
  'seek': 0,
  'start': 5.0200000000000005,
  'end': 5.36,
  'text': ' One second.',
  'tokens': [50616, 1485, 1150, 13, 50633],
  'temperature': 0.0,
  'avg_logprob': -0.35171220952814275,
  'compression_ratio': 1.2300884955752212,
  'no_speech_prob': 0.35816094279289246}]

## Distil Whisper

In [ ]:
!pip install --upgrade pip --quiet
!pip install --upgrade transformers accelerate --quiet

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-large-v2"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    # stride_length_s=10
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
result = pipe('audio.mp3')

In [ ]:
print(f"Transcription:\n\n {result['text'][:2000]}\n\n\n{result['text'][-2000:]}")

Transcription:

  you you you I'm sorry, God. Rahmman, Rahm the haze if it is the tel. Allahuma, ahem! Allah! Allah! Allah! Welcome to everybody. We also have some special guests. I believe we have the oldest one, hiding behind here. Emma, you can run but you can't hide and the lads are here and where's my guy, Idriss? Masha Allah. All right folks so today we start a new chapter as Jazad has just put on the link there. You don't have right click. You mean. Throw your Mac in the bin. Which human being, Yan'i, on his trackpad doesn't make the right-hand side right-click of a mouse? Tell me that. All right. We're going to have a vote right now. Vote right now. He just said that on a track pad, forget Mac, Apple. I'm just talking about human civilization. If you have a track pad, or if you have two buttons, no, no, that's not right. One second, let's start where we should start. If we have a left button and a right button, what would the right button do? Right click as a normal windows? Ok

In [ ]:
# def export_text_to_file(text, filename):
#   """Exports the given text to the given file and force ascii."""
#   text = text.encode("ascii", errors="ignore").decode()
#   with open(filename, 'w') as f:
#     f.write(text)
export_text_to_file(result['text'],"transcript2.txt")

In [ ]:
result = pipe('audio.mp3', return_timestamps=True)

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [ ]:
result.keys()

dict_keys(['text', 'chunks'])

In [ ]:
result['text'][:100]

" you know, I'm sorry, Godine Rauchim, Rahmem, al-Qaeda. Thank you. God bless Muhammad and ala his an"

In [ ]:
result['chunks'][:5]

[{'timestamp': (0.0, 67.0),
  'text': " you know, I'm sorry, Godine Rauchim, Rahmem, al-Qaeda."},
 {'timestamp': (67.0, 86.0),
  'text': " Thank you. God bless Muhammad and ala his and his their own. Allahma no sahile, that's it is tell."},
 {'timestamp': (86.0, 89.0),
  'text': ' And you tel the hazen if she is the the.'},
 {'timestamp': (89.0, 91.0), 'text': " Allahum, I ain't ala d'eckraheck"},
 {'timestamp': (91.0, 110.4),
  'text': " thee, the the the the the the the thea can run but you can't hide and the lads are here and where's my guy"}]

In [ ]:
# ts = result['chunks'][0]['timestamp']
ts[0]

0.0

In [ ]:
dec = ts[0]
dec // 3600

0.0

In [ ]:
def convert_to_ts(decimal):

    # decimal = float(decimal)
    # decimal = int(decimal)
    hours = decimal // 3600
    hours = int(hours)
    minutes = int((decimal % 3600) // 60)
    seconds = int(decimal % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def store_segment(segments):
    for el in segments:
        ts = el['timestamp']
        start = ts[0]
        if(type(start) != 'float'):
            print(start, type(start))
            return
        # print(type(ts), ts, type(ts[0]), ts[1])
        start = convert_to_ts(ts[0])
        end = convert_to_ts(ts[1])
        line = f"[{start} - {end}] {el['text']}"
        # write line to text file
        with open('transcript_ts.txt', 'a', encoding="utf-8") as f:
            f.write(line + '\n')
store_segment(result['chunks'])

0.0 <class 'float'>


<>:15: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:15: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-51-7e31c5d7a50e>:15: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if(type(start) is not 'float'):


## Fast Whisper

In [ ]:
!pipx install -q insanely-fast-whisper

## YT Transcript

In [ ]:
!pip install -q youtube_dl youtube_transcript_api
import youtube_dl
from youtube_transcript_api import YouTubeTranscriptApi

# def download_video_and_transcript(url):
#     # Download video
#     # ydl_opts = {}
#     # with youtube_dl.YoutubeDL(ydl_opts) as ydl:
#     #     ydl.download([url])



In [ ]:
!pip install -q langchain pytube
from langchain.document_loaders import YoutubeAudioLoader, YoutubeLoader
# import youtube_dl
# from youtube_transcript_api import YouTubeTranscriptApi

url = "https://www.youtube.com/watch?v=0r_gtG26iAo"
loader = YoutubeLoader.from_youtube_url(url,add_video_info=True)
docs = loader.load()
# response = docs[0].dict()

In [ ]:
url = "https://www.youtube.com/watch?v=XorjY6kgTLM"

# Get transcript
transcript = YouTubeTranscriptApi.get_transcript(
    'XorjY6kgTLM',
    preserve_formatting=False)
print(transcript)

In [ ]:
docs

In [ ]:
doc = ''
for item in transcript:
    end = item['start'] + item['duration']
    doc += f"[{item['start']} - {end:.2f}] {item['text']}\n"
print(doc)

## Miscellaneous

In [ ]:
# !yt-dlp --no-config https://www.youtube.com/watch?v=2-ZlTc6OniY -k

In [ ]:
URL = "https://www.youtube.com/watch?v=vVXJhrrQGnU"
ID = "vVXJhrrQGnU"

In [ ]:
!pip install -q langchain youtube_transcript_api pytube

In [ ]:
from langchain.document_loaders import YoutubeAudioLoader, YoutubeLoader
loader = YoutubeLoader(ID,add_video_info=True)
docs = loader.load()

In [ ]:
!pip install -q tiktoken sentence_transformers
from langchain.text_splitter import TextSplitter, TokenTextSplitter, SentenceTransformersTokenTextSplitter, RecursiveCharacterTextSplitter
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
st_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0)


In [ ]:
URL = "https://www.youtube.com/watch?v=h-KQjQGb0d4"
loader = YoutubeLoader.from_youtube_url(URL,add_video_info=True)
docs = loader.load()
splits = loader.load_and_split(text_splitter)
sentences = loader.load_and_split(st_splitter)
test = st_splitter.split_documents(docs)

In [ ]:
splits[0]

In [ ]:
print(f"Token:{len(splits)}\tSentences:{len(sentences)}\tSecond:{len(test)}")

In [ ]:
for d in enumerate(splits):
  print(d[1].dict())

In [ ]:
# t1 = "\n\n".join(d[1].dict().page_content for d in enumerate(splits))
t1 = "\n\n".join(d[1].page_content for d in enumerate(sentences))
print(t1)

In [ ]:
!pip install -q llama_hub
from llama_hub.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()

In [ ]:
URL = "https://www.youtube.com/watch?v=XorjY6kgTLM"
documents = loader.load_data(ytlinks=[URL],languages=['en','ar'])

In [ ]:
doc = documents[0].dict()
doc.keys()
for key in doc.keys():
  if key != 'text':
    print(f'{key} = {doc[key]}')
print(doc['text'])

In [ ]:
res = docs[0].dict()
print(f"Length:{len(res['page_content'])}\nType:{res['type']}\nMetadata:{res['metadata']}")

In [ ]:
%%writefile requirements.txt
langchain
pytube
youtube_transcript_api

In [ ]:
! pip freeze -r requirements.txt >> freeze.txt

In [ ]:
# Download a static FFmpeg build and add it to PATH.
def run_ffmpeg():
  exist = !which ffmpeg
  if not exist:
    !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
      && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
    ffmdir = !find . -iname ffmpeg-*-static
    path = %env PATH
    path = path + ':' + ffmdir[0]
    %env PATH $path
  print('')
  !which ffmpeg
  print('Done!')

In [ ]:
import subprocess
def list():
  out = subprocess.run(["ls", "-alh"], capture_output=True)
  print(out.stdout.decode())

In [ ]:
! pip install  -q -U openai-whisper cohere openai
import whisper

In [ ]:
result.keys()

In [ ]:
import openai

In [ ]:
import re
def extract_youtube_video_id(url):
  regex = r"v=([^&]+)"
  match = re.search(regex, url)
  if match:
    return match.group(1)
  else:
    return None

In [ ]:
extract_youtube_video_id(URL)

In [ ]:
extract_youtube_video_id("https://www.youtube.com/watch?v=5tsbWHgBYpw")

In [ ]:
# med_model = whisper.load_model("medium")
small_model = whisper.load_model("small")

In [ ]:
result['text'][:600]

In [ ]:
export_text_to_file(result['text'],"transcript.txt")

In [ ]:
result['text'][:500]

In [ ]:
result['segments'][0]

In [ ]:
import json
res = json.dumps(result['segments'])

In [ ]:
res2 = json.load(res)

In [ ]:
export_text_to_file(res,"segments.txt")

In [ ]:
! pip install -q pytube
import pytube as pt

In [ ]:
def download(url, filename):
  yt = pt.YouTube(url)
  stream = yt.streams.filter(only_audio=True)
  stream[0].download()

In [ ]:
list()

In [ ]:
yor_skit = "https://www.youtube.com/watch?v=YnBUKT7Le8I"
download(yor_skit,"skit")

In [ ]:
def download(url):
  id = extract_youtube_video_id(url)
  cmds = ["yt-dlp", "--no-config", "-v", "--get-title", "--extract-audio", "--audio-format", "mp3", id, ">>", "audio.mp3"]
  out = subprocess.run(cmds, capture_output=True)
  # subprocess.run(["yt-dlp", "--no-config", "-v", "--get-title", "--extract-audio", "--audio-format", "mp3", id, ">>", "audio.mp3"])
  print(str(cmds))
  print(out.stdout.decode())

In [ ]:

yt = pt.YouTube(lp2_url)
stream = yt.streams.filter(only_audio=True)
stream[0].download(filename="audio2.mp3")

In [ ]:
lp2_url = "https://www.youtube.com/watch?v=EXALI6jFu6E"
yt = pt.YouTube(lp2_url)
stream = yt.streams.filter(only_audio=True)
stream[0].download(filename="audio2.mp3")

In [ ]:
%%time
result2 = small_model.transcribe("audio2.mp3")

In [ ]:
result1.keys()

In [ ]:
export_text_to_file(result1["text"],"medium.txt")

In [ ]:
export_text_to_file(result2["text"],"large.txt")

In [ ]:
def highlight_differences(text1, text2):
  """Highlights the differences between two texts.

  Args:
    text1: The first text.
    text2: The second text.

  Returns:
    A string containing the two texts with the differences highlighted.
  """

  # Split the texts into words.
  words1 = text1.split()
  words2 = text2.split()

  # Create a list to store the highlighted differences.
  highlighted_differences = []

  # Iterate over the words in the first text.
  for i in range(len(words1)):
    # If the word in the first text is different from the word in the second text,
    # highlight it.
    if words1[i] != words2[i]:
      # highlighted_differences.append(f"<span style='background-color: yellow'>{words1[i]}</span>")
      highlighted_differences.append(f"[<{words1[i]}><{words2[i]}>]")
    else:
      highlighted_differences.append(words1[i])

  # Join the highlighted differences into a string.
  highlighted_text = " ".join(highlighted_differences)

  # Return the highlighted text.
  return highlighted_text


# Example usage:

text1 = "This is the first text."
text2 = "This is the second text."

highlighted_text = highlight_differences(text1, text2)

print(highlighted_text)


In [ ]:
res1 = result1["text"]
res2 = result2["text"]

In [ ]:
highlight_differences(res1, res2)

In [ ]:
res1

In [ ]:
res2

In [ ]:
!pip uninstall -q pytube
!pip install -q pytube==15.0.0

In [ ]:
eng_arabic_url = "https://www.youtube.com/watch?v=X1mC1XY65Kc&ab_channel=LearnArabicwithKhasu"

In [ ]:
import pytube as pt
yt = pt.YouTube(eng_arabic_url)
streams = yt.video_id
print(yt.streaming_data)
# stream = yt.streams.filter(only_audio=True)
# stream[0].download(filename="mixed.mp3")

In [ ]:
%%time
mixed = small_model.transcribe("mixed.mp3")

In [ ]:
export_text_to_file(mixed['text'], "mixed.txt")

In [ ]:
!pip install -q yt_dlp pydub librosa langchain

In [ ]:
from langchain.document_loaders.generic import GenericLoader
# from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [ ]:
%%time
lp2_url = "https://www.youtube.com/watch?v=EXALI6jFu6E"

ytloader = YoutubeAudioLoader([lp2_url], './')

In [ ]:
ytloader.save_dir

In [ ]:
!pip install -q pytube
import pytube as pt
yt = pt.YouTube(lp2_url)
# stream = yt.streams.filter(only_audio=True)
# stream[0].download(filename="audio2.mp3")

In [ ]:
!pip install -q -U tube_dl
from tube_dl import Youtube
Youtube(lp2_url).formats.filter(only_audio=True)[0].download(convert='mp3')

In [ ]:
%%time
lp2_url = "https://www.youtube.com/watch?v=EXALI6jFu6E"
Youtube(lp2_url).formats.filter(only_audio=True)[0].download(convert='mp3')

In [ ]:
!pip install -q langchain==0.0.304 yt_dlp transformers torch pydub librosa
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers.audio import OpenAIWhisperParserLocal
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [ ]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")

In [ ]:
english_ids = processor.get_decoder_prompt_ids(language="english",task="transcribe")
arabic_ids = processor.get_decoder_prompt_ids(language="arabic",task="transcribe")

In [ ]:
enAudioParse = OpenAIWhisperParserLocal(lang_model="openai/whisper-medium", forced_decoder_ids=english_ids)
arAudioParse = OpenAIWhisperParserLocal(lang_model="openai/whisper-medium", forced_decoder_ids=arabic_ids)

In [ ]:
enAudioParse

In [ ]:
arAudioParse

In [ ]:
urls = ["https://www.youtube.com/watch?v=BqsDOFcr1SU&ab_channel=AlHadyAlNabawy"]
audioLoader = YoutubeAudioLoader(urls, "./")

In [ ]:
enLoader = GenericLoader(audioLoader, enAudioParse)

In [ ]:
docs = enLoader.load()

In [ ]:
def write_file(text, filename):
    with open(filename, "w") as f:
        f.write(text)

In [ ]:
write_file(docs[0].page_content,"LP4en.txt")

In [ ]:
arLoader = GenericLoader(audioLoader, arAudioParse)

In [ ]:
arDocs = arLoader.load()

In [ ]:
arDocs[0].page_content

In [ ]:
!huggingface-cli login

In [ ]:
!pip install -qqq transformers
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")